In [2]:
:dep ndarray
use ndarray::array;
use ndarray::ArrayBase

In [3]:
let a1 = array![1, 2, 3, 4];
a1

[1, 2, 3, 4], shape=[4], strides=[1], layout=CFcf (0xf), const ndim=1

In [4]:
:t a1

a1: ArrayBase<ndarray::OwnedRepr<i32>, ndarray::Dim<[usize; 1]>>

In [5]:
let y = array![[1, 2, 3],
                [4, 5, 6],
                [7,8,9]];
y

[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9]], shape=[3, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2

In [6]:
array!["a", "bcc", "c"]

["a", "bcc", "c"], shape=[3], strides=[1], layout=CFcf (0xf), const ndim=1

In [7]:
let a3 = array![[[1, 2], [3, 4]],
                [[5, 6], [7, 8]]];
a3

[[[1, 2],
  [3, 4]],

 [[5, 6],
  [7, 8]]], shape=[2, 2, 2], strides=[4, 2, 1], layout=Cc (0x5), const ndim=3

In [11]:
use ndarray::{arr2, Axis, concatenate, stack};
let a4 = stack(Axis(0), &[a3.view(), a3.view()]).unwrap();
a4

[[[[1, 2],
   [3, 4]],

  [[5, 6],
   [7, 8]]],


 [[[1, 2],
   [3, 4]],

  [[5, 6],
   [7, 8]]]], shape=[2, 2, 2, 2], strides=[8, 4, 2, 1], layout=Cc (0x5), const ndim=4

In [12]:
:dep pyo3 = {version="0.19.2", features=["auto-initialize"]}
:dep numpy

In [13]:
use numpy::pyo3::Python;
use numpy::pyo3::prelude::*;
use numpy::ndarray::array;
use numpy::{ToPyArray, PyArray};

In [14]:
Python::with_gil(|py| -> PyResult<()> {
    let py_array = array![[1i64, 2], [3, 4]].to_pyarray(py);
    println!("{:?}", py_array);
   //let arr = py_array.readonly().as_array();    
    dbg!(py_array.readonly());
    Ok(())
});

The type of the variable a4 was redefined, so was lost.
The type of the variable a3 was redefined, so was lost.
The type of the variable a1 was redefined, so was lost.
The type of the variable y was redefined, so was lost.
[src/lib.rs:20] py_array.readonly() = PyReadonlyArray<i64, ndarray::dimension::dim::Dim<[usize; 2]>>


array([[1, 2],
       [3, 4]], dtype=int64)


In [15]:
{
let arg1 = "...arg1...";
let arg2 = "arg2";
let arg3 = "arg3";
let mut x = String::from("");

Python::with_gil(|py| -> PyResult<()> {
    let args = (arg1, arg2, arg3);
    x =  PyModule::from_code(
        py,
        "def example(*args, **kwargs):
            return ', '.join(args)",
        "",
        "",
    )?
    .getattr("example")?
        .call1(args)?
        .extract()?;
    dbg!(&x);
    Ok(())
});
println!("jfkdslajf:{x}");
}

[src/lib.rs:50] &x = "...arg1..., arg2, arg3"


()

jfkdslajf:...arg1..., arg2, arg3


In [16]:
use pyo3::prelude::*;
let mut r2 = vec![1,2,3];
Python::with_gil(|py| -> PyResult<()> {
    let result = py
        .eval("[i * 10 for i in range(5)]", None, None)?;
    r2 = result.extract().unwrap();
    dbg!(&r2);
    Ok(())
})

[src/lib.rs:127] &r2 = [
    0,
    10,
    20,
    30,
    40,
]


Ok(())

In [16]:
r2

[0, 10, 20, 30, 40]

In [18]:
use pyo3::prelude::*;
let c = Python::with_gil(|py| -> PyResult<Vec<i32>> {
    Ok(py.eval("[1+1,2+2,3]", None, None)?.extract()?)
})?;
c

[2, 4, 3]

In [19]:
:t c

c: Vec<i32>

In [20]:
use pyo3::prelude::*;
use pyo3::types::IntoPyDict;
Python::with_gil(|py| -> PyResult<()> {
        let sys = py.import("sys")?;
        let version: String = sys.getattr("version")?.extract()?;

        let locals = [("os", py.import("os")?)].into_py_dict(py);
        let code = "os.getenv('USER') or os.getenv('USERNAME') or 'Unknown'";
        let user: String = py.eval(code, None, Some(&locals))?.extract()?;

        println!("Hello {}, I'm Python {} ", user, version);
        Ok(())
    })

Hello tzerj, I'm Python 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)] 


Ok(())

In [21]:

fn python<F>(f2: F) -> PyResult<()> where F: Fn(Python<'_>)-> PyResult<()> {
Python::with_gil(f2)
}
python(|py| {        
        let locals = [("np", py.import("numpy")?)].into_py_dict(py);
        let code = "np.sin(100)";
        let rtn: f64 = py.eval(code, None, Some(&locals))?.extract()?;

        println!("np.sin(100)={rtn}");
        Ok(())
    }
)

np.sin(100)=-0.5063656411097588


Ok(())

In [22]:
use numpy::PyArray1;
use pyo3::{types::IntoPyDict, PyResult, Python};

python(|py| {
        let np = py.import("numpy")?;
        let locals = [("np", np)].into_py_dict(py);

        let pyarray: &PyArray1<i32> = py
            .eval("np.absolute(np.array([-1, -2, -3], dtype='int32'))", Some(locals), None)?
            .extract()?;

        let readonly = pyarray.readonly();
        dbg!(&readonly);
        let slice = readonly.as_slice()?;
        assert_eq!(slice, &[1, 2, 3]);

        Ok(())
    })


[src/lib.rs:155] &readonly = PyReadonlyArray<i32, ndarray::dimension::dim::Dim<[usize; 1]>>


Ok(())